# Analyse d'une collecte de mémoire vive

In [183]:
from colorama import init, Fore, Back, Style
import os
import json
import pandas as pd
import re
from IPython.display import display, HTML

from pandas.io.json import json_normalize
import ipaddress as ip
from msticpy.transform.iocextract import IoCExtract
import msticpy as mp
mp.init_notebook(globals(), verbosity=0)
ti = mp.TILookup()
ioc_extract = IoCExtract()

#Expand the width of the cells
display(HTML("<style>.container { width:90% !important; }</style>"))

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def ip_type(string):
    if ip.ip_address(string).is_private:
        return 'Private'
    elif ip.ip_address(string).is_multicast:
        return 'Multicast'
    elif ip.ip_address(string).is_reserved:
        return 'Reserved'
    elif ip.ip_address(string).is_loopback:
        return 'Loopback'
    elif ip.ip_address(string).is_global:
        return 'Public'
    elif ip.ip_address(string).is_link_local:
        return 'Link local'


mem_path = "/home/secubian/Desktop/Cases/0C-002-memdump.mem"
vol  = f"vol --offline -f {mem_path}"


try_azcli_login=True;enable_add_items_to_help=False
{'try_azcli_login': 'True', 'enable_add_items_to_help': 'False'}
try_azcli_login=True;enable_add_items_to_help=False


## Récupération d'informations relatives au prélévement mémoire

Cette étape est importante pour la suite de l'investigation.
Elle va permettre de vérifier que le profile correspondant au système d'exploitation du prélévement est bien pris en charge par Volatility3. 

In [144]:
os.system(f"{ vol } windows.info.Info")

Volatility 3 Framework 2.4.0



Variable	Value

Kernel Base	0x82a0f000
DTB	0x185000
Symbols	file:///home/secubian/.local/lib/python3.9/site-packages/volatility3/symbols/windows/ntkrpamp.pdb/684DA42A30CC450F81C535B4D18944B1-2.json.xz
Is64Bit	False
IsPAE	True
layer_name	0 WindowsIntelPAE
memory_layer	1 FileLayer
KdDebuggerDataBlock	0x82b39c28
NTBuildLab	7601.17514.x86fre.win7sp1_rtm.10
CSDVersion	1
KdVersionBlock	0x82b39c00
Major/Minor	15.7601
MachineType	332
KeNumberProcessors	1
SystemTime	2019-07-31 13:45:40
NtSystemRoot	C:\Windows
NtProductType	NtProductWinNt
NtMajorVersion	6
NtMinorVersion	1
PE MajorOperatingSystemVersion	6
PE MinorOperatingSystemVersion	1
PE Machine	332
PE TimeDateStamp	Sat Nov 20 08:42:49 2010


0

# Extraction des informations pour analyse

Cette étape va permettre d'exécuter un certain nombre de modules pour extraire l'ensemble des informations nécessaires à l'investigation de l'analyste.

In [2]:
volplugins_lst = ["windows.sessions.Sessions","windows.pslist.PsList","windows.psscan.PsScan","windows.cmdline.CmdLine","windows.netscan.NetScan","windows.callbacks.Callbacks","windows.driverscan.DriverScan","windows.dlllist.DllList","windows.svcscan.SvcScan","windows.handles.Handles","windows.malfind.Malfind","windows.ssdt.SSDT"]

for plugin in volplugins_lst:
    # Check if plugin is already executed
    if not (os.path.isfile(f"{ mem_path }_{ plugin }.json")):
        # Execute plugin and save result in json file
        os.system( f"{ vol } -r json  { plugin } > { mem_path }_{ plugin }.json")

    globals()[f"df_{ plugin.replace('.','_') }"] = pd.read_json(f"{ mem_path }_{ plugin }.json")



## Enrichissement des données collectées

In [184]:
dfsession = df_windows_sessions_Sessions.rename(columns={'Process ID': 'PID'})
#dfsession[['PID', 'User Name']]

# Enrichissement des processus - PsList
df_windows_pslist_PsList_enriched = df_windows_pslist_PsList.merge(dfsession, on='PID', how='left' )
dfppid = df_windows_pslist_PsList_enriched[['PID', 'ImageFileName','User Name']].drop_duplicates()
dfppid.columns = ['PPID', 'ParentName','ParentUserName']
df_windows_pslist_PsList_enriched = df_windows_pslist_PsList_enriched.merge(dfppid, on='PPID', how='left')
#df_windows_pslist_PsList_enriched.sort_values(by=['SessionId','PPID','ParentName'])[['SessionId','PID','ImageFileName','User Name','PPID','ParentName','ParentUserName']].head()


# Enrichissement des processus - PsScan
df_windows_psscan_PsScan_enriched = df_windows_psscan_PsScan.merge(dfsession, on='PID', how='left' )
dfppid = df_windows_psscan_PsScan_enriched[['PID', 'ImageFileName','User Name']].drop_duplicates()
dfppid.columns = ['PPID', 'ParentName','ParentUserName']
df_windows_psscan_PsScan_enriched = df_windows_psscan_PsScan_enriched.merge(dfppid, on='PPID', how='left')
#df_windows_psscan_PsScan_enriched.sort_values(by=['SessionId','PPID','ParentName'])[['SessionId','PID','ImageFileName','User Name','PPID','ParentName','ParentUserName']].head()


# Enrichissement des résultats du mdule CmdLine
df_windows_cmdline_CmdLine_enriched = df_windows_cmdline_CmdLine.merge(dfsession, on='PID', how='left' )
df_windows_cmdline_CmdLine_enriched = df_windows_cmdline_CmdLine_enriched.rename(columns={'Process_x': 'Process'})
dfppid = df_windows_psscan_PsScan_enriched[['PID', 'PPID', 'ParentName','ParentUserName']].drop_duplicates()
df_windows_cmdline_CmdLine_enriched = df_windows_cmdline_CmdLine_enriched.merge(dfppid, on='PID', how='left' )

# Enrichissement des Dlls
df_windows_dlllist_DllList_enriched = df_windows_dlllist_DllList.merge(dfsession, on='PID', how='left' )
dfppid = df_windows_psscan_PsScan_enriched[['PID','PPID', 'ParentName','ParentUserName']].drop_duplicates()
df_windows_dlllist_DllList_enriched = df_windows_dlllist_DllList_enriched.merge(dfppid, on='PID', how='left')
df_windows_dlllist_DllList_enriched = df_windows_dlllist_DllList_enriched[['Path','PID','Process_x','User Name','PPID', 'ParentName','ParentUserName']][df_windows_dlllist_DllList_enriched["Path"].str.contains('programdata|users|utilisateurs|temp|tmp', flags=re.IGNORECASE) == True].drop_duplicates()

# Enrichissement des Handles
df_windows_handles_Handles_enriched = df_windows_handles_Handles.merge(dfsession, on='PID', how='left')
dfppid = df_windows_psscan_PsScan_enriched[['PID','PPID', 'ParentName','ParentUserName']].drop_duplicates()
df_windows_handles_Handles_enriched = df_windows_handles_Handles_enriched.merge(dfppid, on='PID', how='left')
df_windows_handles_Handles_enriched = df_windows_handles_Handles_enriched[['Create Time', 'PID', 'Process_x', 'User Name', 'Name', 'Type',  'GrantedAccess','HandleValue', 'Offset',    'PPID', 'ParentName', 'ParentUserName']]

# Enrichissement des résultats du module : NetScan
df_windows_netscan_NetScan_enriched = df_windows_netscan_NetScan.merge(dfsession, on='PID', how='left')
dfppid = df_windows_psscan_PsScan_enriched[['PID','PPID', 'ParentName','ParentUserName']].drop_duplicates()
df_windows_netscan_NetScan_enriched = df_windows_netscan_NetScan_enriched.merge(dfppid, on='PID', how='left')

# Enrichissement des résultats du module : malfind
df_windows_malfind_Malfind_enriched = df_windows_malfind_Malfind.merge(dfsession[['PID','User Name']], on='PID', how='left')
dfppid = df_windows_psscan_PsScan_enriched[['PID','PPID', 'ParentName','ParentUserName']].drop_duplicates()
df_windows_malfind_Malfind_enriched = df_windows_malfind_Malfind_enriched.merge(dfppid, on='PID', how='left')


# Analyse des processus

Cette phase permet d'analyser les processus en cours d'exécution lors du prélévement.
La première étape consiste donc à analyser la double liste chainée répertoriant les processus au sein du système d'exploitation Microsoft Windows.

## Identification de processus cachés

Analyse les différences entre les résultats des plugins PsTree et PsScan.
Malheureusement, il est tout à fait possible de masquer une exécution de processus malveillant, en ne l'intégrant pas dans cette liste. Il va donc faloir utiliser le module ```PsScan``` qui a pour objectif de scanner l'ensemble de la mémoire à la recherche de structure EPROCESS, puis comparer les deux résultats.

In [185]:
for proc in df_windows_psscan_PsScan_enriched['PID']:
    p_count = df_windows_pslist_PsList[df_windows_pslist_PsList.PID == proc].shape[0]
    if p_count == 0 :
        procPID = proc
        procPPID = df_windows_psscan_PsScan_enriched[df_windows_psscan_PsScan_enriched.PID == proc]['PPID'].to_string(index = False)
        procFileName = df_windows_psscan_PsScan_enriched[df_windows_psscan_PsScan_enriched.PID == proc]['ImageFileName'].to_string(index = False)
        procUserName = df_windows_psscan_PsScan_enriched[df_windows_psscan_PsScan_enriched.PID == proc]['User Name'].to_string(index = False)
        procParentFileName = df_windows_psscan_PsScan_enriched[df_windows_psscan_PsScan_enriched.PID == proc]['ParentName'].to_string(index = False)
        procParentUserName = df_windows_psscan_PsScan_enriched[df_windows_psscan_PsScan_enriched.PID == proc]['ParentUserName'].to_string(index = False)
        print(Fore.RED + f"[!] Suspicious process hidden in psList for PID: { procPID }, Filename: { procFileName}, executed by Username: {procUserName}, with PPID: {procPPID}, ParentFileName: {procParentFileName} executed by: {procParentUserName}")

[!] Suspicious process hidden in psList for PID: 1876, Filename: RGoEsDNcZhEnl., executed by Username: NaN, with PPID: 1416, ParentFileName: rad5163B.tmp.e executed by: johnoc-PC/johnoc
[!] Suspicious process hidden in psList for PID: 2652, Filename: dllhost.exe, executed by Username: NaN, with PPID: 636, ParentFileName: svchost.exe executed by: WORKGROUP/JOHNOC-PC$
[!] Suspicious process hidden in psList for PID: 4076, Filename: chrome.exe, executed by Username: NaN, with PPID: 724, ParentFileName: chrome.exe executed by: None
[!] Suspicious process hidden in psList for PID: 3412, Filename: dllhost.exe, executed by Username: NaN, with PPID: 636, ParentFileName: svchost.exe executed by: WORKGROUP/JOHNOC-PC$
[!] Suspicious process hidden in psList for PID: 1860, Filename: conhost.exe, executed by Username: NaN, with PPID: 420, ParentFileName: csrss.exe executed by: /SYSTEM
[!] Suspicious process hidden in psList for PID: 4012, Filename: tior.exe, executed by Username: NaN, with PPID: 75

## Recherche des processus systèmes ayant plus occurrences

- Définition des processus devant être unique sur un système Microsoft Windows 

In [187]:
singleprocs = ["system","wininit.exe","lsass.exe","lsm.exe","services.exe","lsaiso.exe"]

- Lancement de la recherche

In [188]:
for procs in singleprocs:
    p_count = df_windows_pslist_PsList[df_windows_pslist_PsList.ImageFileName.str.lower() == procs].shape[0]
    if p_count == 1:
        print(Fore.GREEN + "[✓]No multiple instance(s) of " + procs + " found")
    elif p_count > 1:
        print(Fore.RED + "[!]Multiple instance(s) of " + procs + " found!")
        print(df_windows_pslist_PsList[df_windows_pslist_PsList.ImageFileName.str.lower() == procs][['PID','ImageFileName','User Name','PPID','ParentName','ParentUserName']].to_string(index = False))


[✓]No multiple instance(s) of system found
[✓]No multiple instance(s) of wininit.exe found
[✓]No multiple instance(s) of lsass.exe found
[✓]No multiple instance(s) of lsm.exe found
[✓]No multiple instance(s) of services.exe found


## Analyse des processus svchost et rundll32

Tous les processus svchost et rundll32 possèdent des arguments dans un fonctionnement normal. \
Voici donc la liste des processus n'ayant pas d'argument et de ce fait considérés comme suspects.

In [57]:
noargs = df_windows_cmdline_CmdLine_enriched[df_windows_cmdline_CmdLine_enriched['Args'].str.contains('svchost.exe$|rundll32.exe$', flags=re.IGNORECASE)][['PID','Process','Args','User Name']]
if (noargs.empty):
    print(Fore.GREEN + "[✓] No svchost.exe or rundll32.exe process identified without commandline arguments")
else:
    print(Fore.RED + "[!] Some svchost.exe or rundll32.exe without command line was identified!")
    print(noargs.to_string(index = False))
    

[✓] No svchost.exe or rundll32.exe process identified without commandline arguments


## Analyse des Dlls

Affichage de l'ensemble des librairies systèmes chargées à partir de répertoires non habituels.

In [60]:
suspdlls = df_windows_dlllist_DllList_enriched[['Path','PID','Process_x','User Name','PPID', 'ParentName','ParentUserName']][df_windows_dlllist_DllList_enriched["Path"].str.contains('.exe$', flags=re.IGNORECASE) == False]
if (suspdlls.empty):
    print(Fore.GREEN + "[✓] No Suspicious DLLs identified")
else:
    print(Fore.RED + "[!] Suspicious DLLs identified")
    print(suspdlls.to_string(index = False))

[!] Suspicious DLLs identified
                                                                                                            Path  PID      Process_x            User Name  PPID   ParentName   ParentUserName
C:\ProgramData\Microsoft\Windows Defender\Definition Updates\{4A914D27-2C33-4FD8-9CFD-A97728ECCE77}\mpengine.dll 2896    svchost.exe WORKGROUP/JOHNOC-PC$   512 services.exe          /SYSTEM
                                                             C:\Users\johnoc\Documents\Imager_Lite_3.1.1\LMS.dll 2108 FTK Imager.exe     johnoc-PC/johnoc   588 explorer.exe johnoc-PC/johnoc
                                                        C:\Users\johnoc\Documents\Imager_Lite_3.1.1\MSVCP100.dll 2108 FTK Imager.exe     johnoc-PC/johnoc   588 explorer.exe johnoc-PC/johnoc
                                                        C:\Users\johnoc\Documents\Imager_Lite_3.1.1\MSVCR100.dll 2108 FTK Imager.exe     johnoc-PC/johnoc   588 explorer.exe johnoc-PC/johnoc
                   

## Analyse hooking SSDT

In [25]:
sus_kernelhook = df_windows_ssdt_SSDT[(df_windows_ssdt_SSDT["Module"].str.match(r"(ntoskrnl1|win32k)",case=False) == False)][["Address","Module","Symbol"]]
if (sus_kernelhook.empty):
    print(Fore.GREEN + "[✓]No suspicious SSDT hooking detected")
else:
    print(Fore.RED + "[!]Suspicious SSDT hooking detected!")
    print(sus_kernelhook.to_string(index = False))

[✓]No suspicious SSDT hooking detected


## Analyse process identified by malfind

In [189]:
procinject1=df_windows_malfind_Malfind_enriched[df_windows_malfind_Malfind_enriched["Hexdump"].str.contains("MZ") == True][['PID','Process','User Name','Start VPN','End VPN','Protection','Disasm', 'PPID', 'ParentName', 'ParentUserName']]
if (procinject1.empty):
    print(Fore.GREEN + "[✓]No MZ header detected in malfind preview output")
else:
    print(Fore.RED + "[!]MZ header detected within malfind preview (Process Injection indicator) \n")
    print(procinject1[['Protection', 'PID','Process','User Name', 'PPID', 'ParentName', 'ParentUserName']].drop_duplicates().to_string(index = False))



[!]MZ header detected within malfind preview (Process Injection indicator) 

            Protection  PID        Process        User Name  PPID  ParentName   ParentUserName
PAGE_EXECUTE_READWRITE 1416 rad5163B.tmp.e johnoc-PC/johnoc  3544 WINWORD.EXE johnoc-PC/johnoc
PAGE_EXECUTE_READWRITE 1808 rfhyMVOQxfc.ex johnoc-PC/johnoc  2728 cscript.exe johnoc-PC/johnoc


## [TODO] Analyse Handle to detect pipe

Les malwares utilisent très souvent des tubes nommés (Pipe Named) afin d'échanger avec le système 

In [21]:
df_windows_handles_Handles_enriched[(df_windows_handles_Handles_enriched["Type"] == "File") & (df_windows_handles_Handles_enriched["Type"].str.match(r"\\Device\\NamedPipe\\().*",case=False) == False)].head()

,Create Time,PID,Process_x,User Name,Name,Type,GrantedAccess,HandleValue,Offset,PPID,ParentName,ParentUserName
19,2019-07-31T11:49:59,4,System,None,\Device\HarddiskVolume1\$Extend\$RmMetadata\$TxfLog\$TxfLogContainer00000000000000000002,File,1180063,80,2246653280,0,NaN,NaN
20,2019-07-31T11:49:59,4,System,None,\Device\Mup,File,1179926,84,2259860184,0,NaN,NaN
21,2019-07-31T11:49:59,4,System,None,\Device\Tcp,File,1180063,88,2261945864,0,NaN,NaN
22,2019-07-31T11:49:59,4,System,None,\Device\clfs\Device\HarddiskVolume1\$Extend\$RmMetadata\$TxfLog\$TxfLog,File,1245599,96,2268344848,0,NaN,NaN
26,2019-07-31T11:49:59,4,System,None,\Device\clfsKtmLog,File,1180063,112,2267835568,0,NaN,NaN


## [TODO]  Syscall analysis

# Network Analysis

## Identification de connexions distantes, potentiellement suspectes

In [208]:
suspnetworkconnections = df_windows_netscan_NetScan_enriched[['Create Time', 'State', 'Proto', 'LocalAddr', 'LocalPort', 'ForeignAddr', 'ForeignPort', 'Session Type', 'PID', 'Process', 'User Name',   'PPID','ParentName', 'ParentUserName']][(df_windows_netscan_NetScan_enriched["State"] == "ESTABLISHED") | (df_windows_netscan_NetScan_enriched["State"] == "CLOSED")].sort_values(by='PID', ascending=False)
suspnetworkconnections['ip_type'] = suspnetworkconnections['ForeignAddr'].apply(ip_type)

if (suspnetworkconnections[(suspnetworkconnections["ip_type"] == 'Public')].empty):
    print(Fore.GREEN + "[✓] No Potentially Suspicious Network connections identified")
else:
    print(Fore.RED + "[!] Potentially Suspicious Network connections identified\n")
    print(suspnetworkconnections[(suspnetworkconnections["ip_type"] == 'Public')].to_string(index = False))

[!] Potentially Suspicious Network connections identified

        Create Time       State Proto      LocalAddr  LocalPort    ForeignAddr  ForeignPort Session Type    PID        Process        User Name   PPID     ParentName   ParentUserName ip_type
2019-07-31T11:56:34 ESTABLISHED TCPv4 172.16.169.167      49852   66.102.1.108          993      Console 3476.0    OUTLOOK.EXE johnoc-PC/johnoc  588.0   explorer.exe johnoc-PC/johnoc  Public
2019-07-31T12:51:36      CLOSED TCPv4           None      49789 54.154.128.160          443         None 2472.0 systeminfo.exe             None 1416.0 rad5163B.tmp.e johnoc-PC/johnoc  Public


Voici une analyse des adresses ip publiques via les bases de Threat Intelligence : 

In [209]:
# source : 
#os.system('export MSTICPYCONFIG="/home/secubian/Desktop/Documentation/JupyterNotebook/msticpyconfig.yaml"')

#df_windows_netscan_NetScan_public_ip = df_windows_netscan_NetScan_enriched[['ForeignAddr']][(df_windows_netscan_NetScan_enriched["ForeignAddr"] != "*") & (df_windows_netscan_NetScan_enriched["ForeignAddr"] != "0.0.0.0") & (df_windows_netscan_NetScan_enriched["ForeignAddr"] != "::")].drop_duplicates()
#df_windows_netscan_NetScan_public_ip_ioc = ioc_extract.extract(data=df_windows_netscan_NetScan_public_ip, columns=['ForeignAddr'])
#df_windows_netscan_NetScan_public_ip_ioc = df_windows_netscan_NetScan_public_ip_ioc[['IoCType','Observable']].rename(columns={'Observable': 'Ioc'})

df_windows_netscan_NetScan_ti = ti.lookup_iocs(data=suspnetworkconnections, ioc_col="ForeignAddr", providers=["VirusTotal", "OTX"])
df_suspnetworkconnections = df_windows_netscan_NetScan_ti[df_windows_netscan_NetScan_ti['Result']==True]
df_suspnetworkconnections = pd.json_normalize(data=df_suspnetworkconnections[['Ioc','Provider','Details']].to_dict(orient='records'))


if (df_suspnetworkconnections.empty):
    print(Fore.GREEN + "[✓] No Potentially Suspicious Network connections identified")
else:
    print(Fore.RED + "[!] Potentially Suspicious Network connections identified")
    display(df_suspnetworkconnections)



Observables processed:   0%|          | 0/28 [00:00<?, ?obs/s]

KeyError: 'ForeignAddr'

## Identification des processus en écoute sur le système

In [41]:
df_windows_netscan_NetScan_enriched[['Create Time', 'State', 'Proto', 'LocalPort', 'PID', 'Process', 'User Name',   'PPID','ParentName', 'ParentUserName']][(df_windows_netscan_NetScan_enriched["State"] == "LISTENING")].sort_values(by='PID', ascending=False)

,Create Time,State,Proto,LocalPort,PID,Process,User Name,PPID,ParentName,ParentUserName
52,2019-07-31T11:50:03,LISTENING,TCPv4,49156,1960.0,svchost.exe,WORKGROUP/JOHNOC-PC$,512.0,services.exe,/SYSTEM
32,2019-07-31T11:50:03,LISTENING,TCPv4,49156,1960.0,svchost.exe,WORKGROUP/JOHNOC-PC$,512.0,services.exe,/SYSTEM
33,2019-07-31T11:50:03,LISTENING,TCPv6,49156,1960.0,svchost.exe,WORKGROUP/JOHNOC-PC$,512.0,services.exe,/SYSTEM
49,2019-07-31T11:50:01,LISTENING,TCPv4,49154,948.0,svchost.exe,WORKGROUP/JOHNOC-PC$,512.0,services.exe,/SYSTEM
50,2019-07-31T11:50:01,LISTENING,TCPv6,49154,948.0,svchost.exe,WORKGROUP/JOHNOC-PC$,512.0,services.exe,/SYSTEM
48,2019-07-31T11:50:01,LISTENING,TCPv4,49154,948.0,svchost.exe,WORKGROUP/JOHNOC-PC$,512.0,services.exe,/SYSTEM
45,2019-07-31T11:50:01,LISTENING,TCPv4,49153,780.0,svchost.exe,NT AUTHORITY/LOCAL SERVICE,512.0,services.exe,/SYSTEM
44,2019-07-31T11:50:01,LISTENING,TCPv6,49153,780.0,svchost.exe,NT AUTHORITY/LOCAL SERVICE,512.0,services.exe,/SYSTEM
43,2019-07-31T11:50:01,LISTENING,TCPv4,49153,780.0,svchost.exe,NT AUTHORITY/LOCAL SERVICE,512.0,services.exe,/SYSTEM
39,2019-07-31T11:50:01,LISTENING,TCPv6,135,728.0,svchost.exe,WORKGROUP/JOHNOC-PC$,512.0,services.exe,/SYSTEM


In [93]:
df_windows_svcscan_SvcScan

,Binary,Display,Name,Offset,Order,PID,Start,State,Type,__children
0,None,Remote Desktop Services,TermService,10453344,316,NaN,SERVICE_DEMAND_START,SERVICE_STOPPED,SERVICE_WIN32_SHARE_PROCESS,[]
1,\Driver\TermDD,Terminal Device Driver,TermDD,10453344,315,NaN,SERVICE_SYSTEM_START,SERVICE_RUNNING,SERVICE_KERNEL_DRIVER,[]
2,\Driver\tdx,NetIO Legacy TDI Support Driver,tdx,10451464,314,NaN,SERVICE_SYSTEM_START,SERVICE_RUNNING,SERVICE_KERNEL_DRIVER,[]
3,None,TDTCP,TDTCP,10461864,313,NaN,SERVICE_DEMAND_START,SERVICE_STOPPED,SERVICE_KERNEL_DRIVER,[]
4,None,TDPIPE,TDPIPE,10461704,312,NaN,SERVICE_DEMAND_START,SERVICE_STOPPED,SERVICE_KERNEL_DRIVER,[]
5,\Driver\tcpipreg,TCP/IP Registry Compatibility,tcpipreg,10451296,311,NaN,SERVICE_AUTO_START,SERVICE_RUNNING,SERVICE_KERNEL_DRIVER,[]
6,None,Microsoft IPv6 Protocol Driver,TCPIP6,10451128,310,NaN,SERVICE_DEMAND_START,SERVICE_STOPPED,SERVICE_KERNEL_DRIVER,[]
7,\Driver\Tcpip,TCP/IP Protocol Driver,Tcpip,10450960,309,NaN,SERVICE_BOOT_START,SERVICE_RUNNING,SERVICE_KERNEL_DRIVER,[]
8,None,TPM Base Services,TBS,10461544,308,NaN,SERVICE_DEMAND_START,SERVICE_STOPPED,SERVICE_WIN32_SHARE_PROCESS,[]
9,None,Telephony,TapiSrv,10461384,307,NaN,SERVICE_DEMAND_START,SERVICE_STOPPED,SERVICE_WIN32_SHARE_PROCESS,[]
